In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!pip install transformers==4.10.0
!pip install pytorch-lightning

shell-init: error retrieving current directory: getcwd: cannot access parent directories: No such file or directory
shell-init: error retrieving current directory: getcwd: cannot access parent directories: No such file or directory
The folder you are executing pip from can no longer be found.
shell-init: error retrieving current directory: getcwd: cannot access parent directories: No such file or directory
shell-init: error retrieving current directory: getcwd: cannot access parent directories: No such file or directory
The folder you are executing pip from can no longer be found.


In [ ]:
import os
import pandas as pd
os.chdir('/content/drive/MyDrive/Hana_project')
from sklearn.model_selection import train_test_split

from utill import set_seed
from utill import data_collator

from arguments import TrainArguments
from corpus import ClassCorpus, ClassificationDataset
from task import ClassificationTask

import torch
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler
from transformers import AutoTokenizer, AutoConfig, AutoModelForSequenceClassification

from pytorch_lightning import Trainer
from pytorch_lightning.callbacks import ModelCheckpoint

args = TrainArguments(
    pretrained_model_name="monologg/koelectra-base-v3-discriminator",
    downstream_corpus_name="hana_power_v1",
    downstream_task_name="document-classification",
    downstream_model_dir="/content/drive/MyDrive/Hana_project/model",
    max_seq_length=128,
    max_query_length=32,
    doc_stride=64,
    batch_size=32,
    learning_rate=5e-5,
    epochs=1,
    seed=2022,
)

#seet 설정
set_seed(args)

set seed: 2022


In [ ]:
#학습 데이터를 생성합니다.

raw_data = pd.read_excel('/content/drive/MyDrive/Hana_project/data/hana_power_v1/Hana_power_v5_raw.xlsx')

raw_data = raw_data.iloc[:,1:]

raw_data = raw_data.dropna()

raw_data['start_position_character'] = raw_data['start_position_character'].astype('int')

match = pd.read_csv('/content/drive/MyDrive/Hana_project/data/Matching.csv', index_col = 0).to_dict()

raw_data['class'] = raw_data['class'].replace(match['최종'])

raw_data = raw_data.sort_values(by = 'fund')

X_train, X_test = train_test_split(raw_data, test_size = 0.3, random_state=2022)

X_train.to_csv('/content/drive/MyDrive/Hana_project/data/hana_power_v1/Hana_power_v1_train_class.csv', encoding = 'cp949')

X_test.to_csv('/content/drive/MyDrive/Hana_project/data/hana_power_v1/Hana_power_v1_dev_class.csv', encoding = 'cp949')

In [ ]:
raw_data['class'].value_counts()

투자신탁재산       2591
수익권과 수익증권     938
판매 및 환매       731
기타            269
Name: class, dtype: int64

In [ ]:
#Tokenizer

tokenizer = AutoTokenizer.from_pretrained(
    args.pretrained_model_name,
    do_lower_case=False,
    use_fast=False
)

In [ ]:
#corpus를 불러옵니다

corpus = ClassCorpus()

train_dataset = ClassificationDataset(
    args=args,
    corpus=corpus,
    tokenizer=tokenizer,
    mode="train",
)

In [ ]:
#DataLoader를 설정합니다.

train_dataloader = DataLoader(
    train_dataset,
    batch_size=args.batch_size,
    sampler=RandomSampler(train_dataset, replacement=False),
    collate_fn=data_collator,
    drop_last=False,
    num_workers=args.cpu_workers,
)

val_dataset = ClassificationDataset(
    args=args,
    corpus=corpus,
    tokenizer=tokenizer,
    mode="test",
)
val_dataloader = DataLoader(
    val_dataset,
    batch_size=args.batch_size,
    sampler=SequentialSampler(val_dataset),
    collate_fn=data_collator,
    drop_last=False,
    num_workers=args.cpu_workers,
)

In [ ]:
#모델을 설정합니다.


pretrained_model_config = AutoConfig.from_pretrained(
    args.pretrained_model_name,
    num_labels=corpus.num_labels,
)
model = AutoModelForSequenceClassification.from_pretrained(
        args.pretrained_model_name,
        config=pretrained_model_config,
)

Some weights of the model checkpoint at monologg/koelectra-base-v3-discriminator were not used when initializing ElectraForSequenceClassification: ['discriminator_predictions.dense_prediction.bias', 'discriminator_predictions.dense_prediction.weight', 'discriminator_predictions.dense.bias', 'discriminator_predictions.dense.weight']
- This IS expected if you are initializing ElectraForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing ElectraForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of ElectraForSequenceClassification were not initialized from the model checkpoint at monologg/koelectra-base-v3-discriminator and are newly initialized: 

In [ ]:
#task를 정의합니다.

task = ClassificationTask(model, args)

In [ ]:
import torch

def get_trainer(args, return_trainer_only=True):

    ckpt_path = os.path.abspath(args.downstream_model_checkpoint_fpath)
    os.makedirs(ckpt_path, exist_ok=True)
    checkpoint_callback = ModelCheckpoint(
        dirpath=ckpt_path,
        save_top_k=args.save_top_k,
        monitor=args.monitor.split()[1],
        mode=args.monitor.split()[0],
        filename='{epoch}-{val_loss:.2f}',
    )
    
    trainer = Trainer(
        max_epochs=args.epochs,
        fast_dev_run=args.test_mode,
        num_sanity_val_steps=None if args.test_mode else 0,
        callbacks=[checkpoint_callback],
        default_root_dir=ckpt_path,

        # For GPU Setup
        deterministic=torch.cuda.is_available() and args.seed is not None,
        gpus=torch.cuda.device_count() if torch.cuda.is_available() else None,
        precision=16 if args.fp16 else 32,
        
        )
    if return_trainer_only:
        return trainer

    else:
        return checkpoint_callback, trainer

In [ ]:
#모델 학습을 진행합니다.

trainer = get_trainer(args)

trainer.fit(task,train_dataloaders=train_dataloader,val_dataloaders=val_dataloader)

/usr/local/lib/python3.7/dist-packages/pytorch_lightning/trainer/connectors/accelerator_connector.py:448: LightningDeprecationWarning: Setting `Trainer(gpus=1)` is deprecated in v1.7 and will be removed in v2.0. Please use `Trainer(accelerator='gpu', devices=1)` instead.
  f"Setting `Trainer(gpus={gpus!r})` is deprecated in v1.7 and will be removed"
INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
/usr/local/lib/python3.7/dist-packages/pytorch_lightning/callbacks/model_checkpoint.py:616: UserWarning: Checkpoint directory /content/drive/MyDrive/Hana_project/model/checkpoint exists and is not empty.
  rank_zero_warn(f"Checkpoint directory {dirpath} exists and is not empty.")
INFO:pytorch_lightning.accelerators.cuda:LO

Training: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

INFO:pytorch_lightning.utilities.rank_zero:`Trainer.fit` stopped: `max_epochs=1` reached.


In [ ]:
from tqdm import tqdm

In [ ]:
def inference_fn(sentences):

    result = []

    for sentence in tqdm(sentences):
      inputs = tokenizer(
          [sentence],
          max_length=args.max_seq_length,
          padding="max_length",
          truncation=True,
      )
      with torch.no_grad():
          outputs = model(**{k: torch.tensor(v) for k, v in inputs.items()})
          prob = outputs.logits.softmax(dim=1)
          result.append(prob.argmax(1))

    return result

In [ ]:
#평가 데이터를 호출합니다.

x_test = pd.read_csv('/content/drive/MyDrive/Hana_project/data/hana_power_v1/Hana_power_v1_dev_class.csv',encoding = 'cp949')

qus = x_test['question_text'].tolist()
cla = x_test['class'].tolist()

In [ ]:
x_test

,Unnamed: 0,content,fund,class,question_text,answer_text,start_position_character
0,2322,신탁계약의 변경) ①집합투자업자는 신탁계약을 변경하고자 하는 경우에는 신탁업자와 변...,유진지수연계리자드증권투자신탁SHE-23호(USD)[ELS-파생형],투자신탁재산,신탁계약의 변경은?,집합투자업자는 신탁계약을 변경하고자 하는 경우에는 신탁업자와 변경계약을 체결하여야 한다.,12
1,2769,집합투자기구의 해지) ①투자신탁을 설정한 집합투자업자는 금융위원회의 승인을 받\n아...,하나UBS Japan 증권 자투자신탁(U-1호)(주식),투자신탁재산,집합투자기구의 해지는 뭐야?,투자신탁을 설정한 집합투자업자는 금융위원회의 승인을 받\n아 투자신탁을 해지할 수 있다,14
2,2932,"수익권의 분할) ①이 투자신탁의 수익권은 1좌의 단위로 균등하게 분할하며, 수익증권...",하나UBS Smart Up Plus포커스포트폴리오증권제4호(주식혼합),수익권과 수익증권,수익권의 분할이란?,"이 투자신탁의 수익권은 1좌의 단위로 균등하게 분할하며, 수익증권으로 발행한다.",11
3,2697,판매수수료) ①판매회사는 Class A 및 Class A-E 수익증권의 판매행위에 ...,유진트리플리자드지수연계투자신탁EHN-7호[ELS-파생형],판매 및 환매,판매수수료란?,판매회사는 Class A 및 Class A-E 수익증권의 판매행위에 대한 대가로 수...,9
4,3163,수익증권의 발행 및 전자등록) ①집합투자업자는 제6조 및 제7조의 규정에 의한 투자...,하나UBS 인Best연금 증권투자신탁(제1호)(국공채),수익권과 수익증권,수익증권의 발행 및 전자등록이란?,집합투자업자는 제6조 및 제7조의 규정에 의한 투자신탁의 최초설정 및 추가설정에 의...,19
...,...,...,...,...,...,...,...
1354,3799,운용 및 투자 제한) 집합투자업자는 투자신탁재산을 운용함에 있어 다음 각 호에 해당...,하나UBS승승장구코리아증권투자신탁[주식],투자신탁재산,운용 및 투자 제한은?,집합투자업자는 투자신탁재산을 운용함에 있어 다음 각 호에 해당하는 행위를 신탁업자에...,13
1355,1401,"집합투자업자 및 신탁업자의 업무) ①집합투자업자는 투자신탁의 설정?해지, 투자신탁재...",삼성배당플러스30증권자투자신탁II 제2호[채권혼합],투자신탁재산,집합투자업자의 업무는?,"집합투자업자는 투자신탁의 설정?해지, 투자신탁재산의 운용?운용지시업무를 수행한다",21
1356,2996,판매가격) ①수익증권의 판매가격은 투자자가 이 수익증권의 취득을 위하여 판매회사에 ...,하나UBS StepUp Plus China증권자투자신탁1호(주식혼합),판매 및 환매,판매가격은 무엇인가?,수익증권의 판매가격은 투자자가 이 수익증권의 취득을 위하여 판매회사에 자금을 납입한...,8
1357,2403,수익증권의 발행 및 예탁) ① 집합투자업자는 제6조의 규정에 의한 수종의 수익증권 ...,유진지수연계리자드증권투자신탁SHE-28호(USD)[ELS-파생형],수익권과 수익증권,수익증권의 발행이란?,집합투자업자는 제6조의 규정에 의한 수종의 수익증권 총좌수 범위 내에서 추가설정을 ...,18


In [ ]:
# 평가 데이터에 대해 예측합니다.

prds =  inference_fn(qus)

100%|██████████| 1359/1359 [06:34<00:00,  3.44it/s]


In [ ]:
#classes = {0:'수익권과 수익증권',1:'투자신탁재산',2:"판매 및 환매", 3:'기타'}

In [ ]:
def compute_exact(a_gold, a_pred):

  classes = {0:'수익권과 수익증권',1:'투자신탁재산',2:"판매 및 환매", 3:'기타'}
  
  a_gold = list(map(classes.get, a_gold.tolist()))[0]

  return int(a_gold == a_pred)

In [ ]:
accuracy = []

for i in tqdm(range(len(prds))):
  accuracy.append(compute_exact(prds[i], cla[i]))

100%|██████████| 1359/1359 [00:00<00:00, 410253.28it/s]


In [ ]:
#모델 평가 점수를 보여줍니다.
sum(accuracy) / len(accuracy)

0.9367181751287712

In [ ]:
# 모델을 저장합니다.

torch.save(model, f'model/model_classification.pt')